# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os

# Set your preferred name
USER = "Jinglin"
# Set the data and output directories
DATA_ROOT = '../../DATA'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Craniosynostosis'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [7]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Bladder_Cancer_(BLCA)',
 '.DS_Store',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Stomach_Cancer_(STAD)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Glioblastoma_(GBM)']

If no match is found, jump directly to GEO in Part 2.2

In [6]:
trait_subdir = "TCGA_Thyroid_Cancer_(THCA)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.THCA.sampleMap_THCA_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.THCA.sampleMap_HiSeqV2_PANCAN.gz')

In [6]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [7]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [8]:
clinical_data_cols[:10]

['_INTEGRATION',
 '_PATIENT',
 '_cohort',
 '_primary_disease',
 '_primary_site',
 'additional_pharmaceutical_therapy',
 'additional_radiation_therapy',
 'additional_surgery_locoregional_procedure',
 'age_at_initial_pathologic_diagnosis',
 'bcr_followup_barcode']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [9]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'age_at_initial_pathologic_diagnosis', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'braf_gene_genotyping_outcome_lab_results_text', 'days_to_additional_surgery_metastatic_procedure', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathol

In [12]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis',
                      'days_to_birth']
candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [13]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [74, 65, 55, 57, 33],
 'days_to_birth': [-27348, -24102, -20361, -21008, -12295]}

In [16]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [17]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['FEMALE', 'MALE', 'MALE', 'MALE', 'FEMALE']}

In [18]:
gender_col = 'gender'

In [19]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [20]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [21]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Thyroid Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-4C-A93U-01,1,74,0,-0.767592,-1.742926,-0.531035,-0.067972,0.265922,-0.13531,0.440006,...,-0.748878,-1.018517,0.205567,-1.28139,0.395523,1.735414,0.945273,-0.186695,4.024767,-0.273612
TCGA-BJ-A0YZ-01,1,65,1,1.766308,0.273374,-0.531035,0.790028,1.209322,0.01229,0.704606,...,0.125322,1.568883,0.626367,2.38101,0.275823,1.820014,2.140073,1.380405,1.257367,-0.174312
TCGA-BJ-A0Z0-01,1,55,1,0.986808,-2.165726,-0.050935,0.051428,0.979522,0.48229,0.090106,...,-0.748878,-0.747417,-0.931133,-1.28139,-0.222077,0.293414,0.725173,-0.254095,0.530667,-0.430812
TCGA-BJ-A0Z2-01,1,57,1,0.767208,-1.623626,-0.531035,0.182228,0.541622,0.65889,0.647206,...,-0.748878,-1.587117,-1.292433,-1.28139,-0.602177,0.134114,0.894473,-0.380395,-2.502833,0.360588
TCGA-BJ-A0Z3-01,1,33,0,-0.287292,-1.766626,-0.531035,0.059628,0.920522,0.21849,0.336106,...,-0.413778,1.111583,1.158867,-1.28139,0.003723,1.240514,0.924373,-0.143495,2.613367,-0.030012


In [22]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 572 samples.


In [23]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Thyroid Cancer', the least common label is '0' with 59 occurrences. This represents 10.31% of the dataset.
The distribution of the feature 'Thyroid Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 34.0
  50% (Median): 46.0
  75%: 58.0
Min: 15
Max: 89
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1' with 157 occurrences. This represents 27.45% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [24]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [25]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Jinglin/Thyroid-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [4]:
dataset = 'GEO'
trait_subdir = "Craniosynostosis"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE27976']

Repeat the below steps for all the accession numbers

In [5]:
cohort = accession_num = "GSE27976"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('../../DATA/GEO/Craniosynostosis/GSE27976/GSE27976_family.soft.gz',
 '../../DATA/GEO/Craniosynostosis/GSE27976/GSE27976_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [6]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"Calvarial osteoblast transcriptome analysis identifies genetic targets and extracellular matrix-mediated focal adhesion as potential biomarkers for single-suture craniosynostosis"
!Series_summary	"Craniosynostosis is a disease defined by premature fusion of one or more cranial sutures. The mechanistic pathology of isolated single-suture craniosynostosis is complex and while a number of genetic biomarkers and environmental predispositions have been identified, in many cases the causes remain controversial and inconclusive at best.  After controlling for variables contributing to potential bias, FGF7, SFRP4, and VCAM1 emerged as potential genetic biomarkers for single-suture craniosynostosis due to their significantly large changes in gene expression compared to the control population.  Furthermore, pathway analysis implicated focal adhesion and extracellular matrix (ECM)-receptor interaction as differentially regulated gene networks when comparing all cases of single-sutu

In [7]:
clinical_data.head()

,!Sample_geo_accession,GSM692146,GSM692147,GSM692148,GSM692149,GSM692150,GSM692151,GSM692152,GSM692153,GSM692154,...,GSM692385,GSM692386,GSM692387,GSM692388,GSM692389,GSM692390,GSM692391,GSM692392,GSM692393,GSM692394
0,!Sample_characteristics_ch1,age months: 12.87,age months: 10.4,age months: 12.3,age months: 11.4,age months: 10.1,age months: 11,age months: 4.27,age months: 7.97,age months: 4.33,...,age months: 84,age months: 13,age months: 2,age months: 0.5,age months: 0.75,age months: 0.57,age months: 25.03,age months: 4.47,age months: 4.23,age months: 10.4
1,!Sample_characteristics_ch1,gender: F,gender: F,gender: M,gender: M,gender: F,gender: M,gender: M,gender: M,gender: F,...,gender: M,gender: M,gender: M,gender: M,gender: M,gender: M,gender: F,gender: M,gender: M,gender: M
2,!Sample_characteristics_ch1,type: Metopic Synostosis,type: Metopic Synostosis,type: Coronal Synostosis R,type: Metopic Synostosis,type: Metopic Synostosis,type: Metopic Synostosis,type: Sagittal Synostosis,type: Sagittal Synostosis,type: Sagittal Synostosis,...,type: Control,type: Control,type: Sagittal Synostosis,type: Control,type: Control,type: Control,type: Control,type: Control,type: Control,type: Control
3,!Sample_characteristics_ch1,cell lines: osteoblast,cell lines: osteoblast,cell lines: osteoblast,cell lines: osteoblast,cell lines: osteoblast,cell lines: osteoblast,cell lines: osteoblast,cell lines: osteoblast,cell lines: osteoblast,...,cell lines: osteoblast,cell lines: osteoblast,cell lines: osteoblast,cell lines: osteoblast,cell lines: osteoblast,cell lines: osteoblast,cell lines: osteoblast,cell lines: osteoblast,cell lines: osteoblast,cell lines: osteoblast
4,!Sample_characteristics_ch1,tissue: skull,tissue: skull,tissue: skull,tissue: skull,tissue: skull,tissue: skull,tissue: skull,tissue: skull,tissue: skull,...,tissue: skull,tissue: skull,tissue: skull,tissue: skull,tissue: skull,tissue: skull,tissue: skull,tissue: skull,tissue: skull,tissue: skull


In [8]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['age months: 12.87',
  'age months: 10.4',
  'age months: 12.3',
  'age months: 11.4',
  'age months: 10.1',
  'age months: 11',
  'age months: 4.27',
  'age months: 7.97',
  'age months: 4.33',
  'age months: 9.33',
  'age months: 7.93',
  'age months: 10.27',
  'age months: 10.87',
  'age months: 3.87',
  'age months: 3.2',
  'age months: 13.27',
  'age months: 5.6',
  'age months: 14.9',
  'age months: 3.03',
  'age months: 12.4',
  'age months: 8.9',
  'age months: 14.17',
  'age months: 6.33',
  'age months: 14.87',
  'age months: 8.4',
  'age months: 9.07',
  'age months: 13.33',
  'age months: 10',
  'age months: 13.23',
  'age months: 10.33'],
 1: ['gender: F', 'gender: M'],
 2: ['type: Metopic Synostosis',
  'type: Coronal Synostosis R',
  'type: Sagittal Synostosis',
  'type: Coronal Synostosis L',
  'type: Control'],
 3: ['cell lines: osteoblast'],
 4: ['tissue: skull']}

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [9]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.
   NOTE: Function "convert_trait" should return 1 or 0.
   
Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Craniosynostosis\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Craniosynostosis\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The ke

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [35]:
age_row = gender_row = None
convert_age = convert_gender = None

In [36]:
is_gene_availabe = True
trait_row = 2

age_row = 0
gender_row = 1

trait_type = 'binary'

In [37]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [38]:
def convert_trait(value):
    """
    Converts a given value to a binary representation for craniosynostosis.
    Assumes craniosynostosis presence is indicated by specific keywords (e.g., types of synostosis).
    Returns 1 for presence, 0 for absence, and None for unknown values.

    :param value: The value to be converted (string).
    :return: Binary representation (1 or 0) or None.
    """
    # Assuming presence of craniosynostosis is indicated by certain keywords
    craniosynostosis_keywords = ['sagittal', 'coronal', 'metopic']
    
    if any(keyword in value.lower() for keyword in craniosynostosis_keywords):
        return 1
    elif 'control' in value.lower():
        return 0
    else:
        return None

# Function to convert age values to continuous
def convert_age(value):
    try:
        # Extracting the numerical part of the age value
        age = float(value.split(': ')[1])
        return age
    except (ValueError, IndexError):
        # If conversion fails or format is unexpected, return None
        return None

# Function to convert gender values to binary
def convert_gender(value):
    # Convert gender to binary, M as 1 and F as 0
    if value == "gender: M":
        return 1
    elif value == "gender: F":
        return 0
    else:
        return None

In [39]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM692146,GSM692147,GSM692148,GSM692149,GSM692150,GSM692151,GSM692152,GSM692153,GSM692154,GSM692155,...,GSM692385,GSM692386,GSM692387,GSM692388,GSM692389,GSM692390,GSM692391,GSM692392,GSM692393,GSM692394
Craniosynostosis,1.00,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,...,0.0,0.0,1.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0
Age,12.87,10.4,12.3,11.4,10.1,11.0,4.27,7.97,4.33,9.33,...,84.0,13.0,2.0,0.5,0.75,0.57,25.03,4.47,4.23,10.4
Gender,0.00,0.0,1.0,1.0,0.0,1.0,1.00,1.00,0.00,1.00,...,1.0,1.0,1.0,1.0,1.00,1.00,0.00,1.00,1.00,1.0


### Genetic data preprocessing and final filtering

In [40]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM692146,GSM692147,GSM692148,GSM692149,GSM692150,GSM692151,GSM692152,GSM692153,GSM692154,GSM692155,...,GSM692385,GSM692386,GSM692387,GSM692388,GSM692389,GSM692390,GSM692391,GSM692392,GSM692393,GSM692394
ID,,,,,,,,,,,,,,,,,,,,,
7892501,7.947704,6.676798,7.843258,6.321158,8.001171,6.948549,6.845051,7.614498,6.022057,7.501036,...,7.340116,6.576175,5.896850,8.151285,7.850658,8.319950,6.557796,6.952909,6.710567,7.381286
7892502,5.720497,5.853777,5.592118,5.358243,5.870100,5.505210,6.317334,5.849439,5.875447,6.162370,...,6.084514,5.738185,5.146971,6.325365,6.339419,5.683592,6.064663,5.688963,5.608666,6.098935
7892503,3.692077,3.784867,3.564129,2.796225,3.101595,3.334568,3.599359,3.861042,3.160540,4.897819,...,4.090935,3.089079,4.097332,3.758362,2.921880,3.968212,4.643388,4.330150,4.137814,5.568669
7892504,9.967597,9.804028,9.837678,9.732824,9.386302,9.655602,9.210620,9.076825,9.091269,9.542767,...,9.819561,9.686096,10.014200,9.600938,9.319043,9.200205,9.751630,10.008660,10.079070,9.826204
7892505,4.004615,2.961565,3.170119,2.939202,3.164947,2.621078,3.695578,2.330928,2.530791,3.602525,...,3.220282,4.483160,4.089536,3.040828,3.294937,2.908012,2.486737,2.756370,3.498060,2.938303


In [41]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['7892501',
 '7892502',
 '7892503',
 '7892504',
 '7892505',
 '7892506',
 '7892507',
 '7892508',
 '7892509',
 '7892510',
 '7892511',
 '7892512',
 '7892513',
 '7892514',
 '7892515',
 '7892516',
 '7892517',
 '7892518',
 '7892519',
 '7892520']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [42]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['7892501', '7892502', '7892503', '7892504', '7892505', '7892506', '7892507', '7892508', '7892509', '7892510', '7892511', '7892512', '7892513', '7892514', '7892515', '7892516', '7892517', '7892518', '7892519', '7892520']\n"

If not required, jump directly to the gene normalization step

In [43]:
requires_gene_mapping = True

In [23]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001004195,NM_001005240,NM_001005484,BC136848,BC136867,BC136907,BC136908', 'NR_024437,XM_006711854,XM_006726377,XR_430662,AK298283,AL137655,BC032332,BC118988,BC122537,BC131690,NM_207366,AK301928,BC071667', 'NM_001005221,NM_001005224,NM_001005277,NM_001005504,BC137547,BC137568'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': ['53049', '63015', '69091', '334129', '367659'], 'RANGE_STOP': ['54936', '63887', '70008', '334296', '368597'], 'total_probes': [7.0, 31.0, 24.0, 6.0, 36.0], 'gene_assignment': ['---', 'ENST00000328113 // OR4G2P // olfactory receptor, family 4, subfamily G, member 2 pseudogene // --- // --- /// ENST000004928

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [24]:
# if requires_gene_mapping:
f'''
As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
probe_id_key = key_name1
gene_symb_key = key_name2

Gene annotation dictionary:
{gene_annotation_summary}
'''

"\nAs a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:\nprobe_id_key = key_name1\ngene_symb_key = key_name2\n\nGene annotation dictionary:\n{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001004195,NM_001005240,NM_001005484,BC136848,BC136867,BC136907,BC136908', 'NR_024437,XM_006711854,XM_006726377,XR_430662,AK298283,AL137655,BC032332,BC118988,BC122537,BC131690,NM_207366,AK301928,BC071667', 'NM_001005221,NM_001005224,NM_001005277,NM_001005504,BC137547,BC137568'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_00

In [44]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'gene_assignment'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [45]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

998 input query terms found no hit:	['---', '---', '1', '1', '1 alpha subcomplex', '1 beta subcomplex', '1 like', '1 pseudogene', '1 pse
1000 input query terms found no hit:	['100507404', '100507424', '100507424', '100507436', '100507436', '100507444', '100507463', '1005074
1000 input query terms found no hit:	['101930567', '101930568', '101930571', '101930578', '101930578', '101930583', '101930591', '1019305
1000 input query terms found no hit:	['10625', '10626', '10626', '10627', '10627', '10628', '10628', '10629', '10629', '1063', '1063', '1
1000 input query terms found no hit:	['1117', '1117', '11170', '11170', '11171', '11171', '11172', '11172', '11173', '11173', '11174', '1
1000 input query terms found no hit:	['1184', '1184', '118421', '118421', '118424', '118424', '118425', '118425', '118426', '118426', '11
1000 input query terms found no hit:	['128360', '128366', '128366', '128367', '128367', '128368', '128368', '128371', '128371', '128372',
1000 input query terms found no hit

913 input query terms found no hit:	['AF493785', 'AF493798', 'AF493869', 'AF493871', 'AF493872', 'AF493873', 'AF493874', 'AF493875', 'AF
998 input query terms found no hit:	['AJ420501', 'AJ420505', 'AJ420506', 'AJ420507', 'AJ420510', 'AJ420514', 'AJ420518', 'AJ420519', 'AJ
1000 input query terms found no hit:	['AK026744', 'AK026746', 'AK026747', 'AK026754', 'AK026760', 'AK026762', 'AK026765', 'AK026786', 'AK
1000 input query terms found no hit:	['AK075051', 'AK075065', 'AK075071', 'AK075095', 'AK075099', 'AK075103', 'AK075115', 'AK075130', 'AK
1000 input query terms found no hit:	['AK094047', 'AK094051', 'AK094052', 'AK094054', 'AK094059', 'AK094062', 'AK094065', 'AK094067', 'AK
999 input query terms found no hit:	['AK098243', 'AK098255', 'AK098260', 'AK098262', 'AK098266', 'AK098267', 'AK098268', 'AK098276', 'AK
1000 input query terms found no hit:	['AK126025', 'AK126030', 'AK126032', 'AK126033', 'AK126034', 'AK126041', 'AK126044', 'AK126046', 'AK
999 input query terms found no hit:	[

1000 input query terms found no hit:	['BC104203', 'BC104206', 'BC104207', 'BC104208', 'BC104212', 'BC104213', 'BC104214', 'BC104216', 'BC
1000 input query terms found no hit:	['BC108666', 'BC108669', 'BC108670', 'BC108675', 'BC108676', 'BC108677', 'BC108680', 'BC108684', 'BC
1000 input query terms found no hit:	['BC112195', 'BC112196', 'BC112197', 'BC112199', 'BC112200', 'BC112201', 'BC112203', 'BC112207', 'BC
1000 input query terms found no hit:	['BC117351', 'BC117354', 'BC117356', 'BC117357', 'BC117358', 'BC117359', 'BC117360', 'BC117362', 'BC
1000 input query terms found no hit:	['BC126402', 'BC126404', 'BC126405', 'BC126406', 'BC126407', 'BC126408', 'BC126409', 'BC126410', 'BC
1000 input query terms found no hit:	['BC132850', 'BC132851', 'BC132852', 'BC132854', 'BC132859', 'BC132860', 'BC132862', 'BC132863', 'BC
1000 input query terms found no hit:	['BC137201', 'BC137202', 'BC137203', 'BC137204', 'BC137205', 'BC137206', 'BC137208', 'BC137209', 'BC
1000 input query terms found no hi

1000 input query terms found no hit:	['ENST00000398165', 'ENST00000398168', 'ENST00000398169', 'ENST00000398174', 'ENST00000398179', 'ENS
1000 input query terms found no hit:	['ENST00000401609', 'ENST00000401611', 'ENST00000401617', 'ENST00000401624', 'ENST00000401630', 'ENS
1000 input query terms found no hit:	['ENST00000406482', 'ENST00000406486', 'ENST00000406487', 'ENST00000406492', 'ENST00000406503', 'ENS
1000 input query terms found no hit:	['ENST00000409816', 'ENST00000409817', 'ENST00000409818', 'ENST00000409820', 'ENST00000409822', 'ENS
1000 input query terms found no hit:	['ENST00000413733', 'ENST00000413748', 'ENST00000413749', 'ENST00000413751', 'ENST00000413755', 'ENS
1000 input query terms found no hit:	['ENST00000416793', 'ENST00000416795', 'ENST00000416796', 'ENST00000416801', 'ENST00000416802', 'ENS
1000 input query terms found no hit:	['ENST00000419892', 'ENST00000419893', 'ENST00000419908', 'ENST00000419909', 'ENST00000419910', 'ENS
1000 input query terms found no hi

1000 input query terms found no hit:	['ENST00000568900', 'ENST00000568901', 'ENST00000568907', 'ENST00000568908', 'ENST00000568909', 'ENS
1000 input query terms found no hit:	['ENST00000572431', 'ENST00000572438', 'ENST00000572447', 'ENST00000572448', 'ENST00000572449', 'ENS
1000 input query terms found no hit:	['ENST00000576362', 'ENST00000576363', 'ENST00000576369', 'ENST00000576370', 'ENST00000576372', 'ENS
1000 input query terms found no hit:	['ENST00000580548', 'ENST00000580551', 'ENST00000580556', 'ENST00000580557', 'ENST00000580558', 'ENS
1000 input query terms found no hit:	['ENST00000584668', 'ENST00000584678', 'ENST00000584689', 'ENST00000584701', 'ENST00000584709', 'ENS
1000 input query terms found no hit:	['ENST00000587671', 'ENST00000587673', 'ENST00000587675', 'ENST00000587678', 'ENST00000587679', 'ENS
1000 input query terms found no hit:	['ENST00000590347', 'ENST00000590348', 'ENST00000590354', 'ENST00000590360', 'ENST00000590361', 'ENS
1000 input query terms found no hi

1000 input query terms found no hit:	['NR_026598', 'NR_026641', 'NR_026643', 'NR_026644', 'NR_026645', 'NR_026648', 'NR_026649', 'NR_0266
1000 input query terms found no hit:	['NR_040718', 'NR_040722', 'NR_040723', 'NR_040724', 'NR_040733', 'NR_040762', 'NR_044998', 'NR_0449
30 input query terms found dup hits:	[('NSL1', 2), ('NSUN5P2', 2), ('NUDT4P2', 2), ('NUDT9P1', 2), ('NUF2', 2), ('NUS1P1', 2), ('NUS1P2',
685 input query terms found no hit:	['NR_103837', 'NR_103842', 'NR_103864', 'NR_103866', 'NR_103867', 'NR_103868', 'NR_103869', 'NR_1038
171 input query terms found dup hits:	[('OR13D3P', 2), ('OR1D3P', 2), ('OR1E3', 2), ('OR1H1P', 2), ('OR1P1', 2), ('OR2A1-AS1', 2), ('OR2A3
83 input query terms found no hit:	['OR13C7P', 'OR14L1P', 'OR1F12', 'OR2A1 antisense RNA 1', 'OR2W5', 'OR4D12P', 'OR51H1P', 'OR52A4', '
66 input query terms found dup hits:	[('PDIA3P1', 2), ('PDXDC2P', 2), ('PERP', 2), ('PFN1P3', 2), ('PFN1P4', 2), ('PFN1P9', 2), ('PGAM1P1
151 input query terms found no hit:	

1000 input query terms found no hit:	['uncharacterized FLJ40288', 'uncharacterized FLJ44385', 'uncharacterized FLJ44790', 'uncharacterize
1000 input query terms found no hit:	['zinc finger protein 141', 'zinc finger protein 142', 'zinc finger protein 143', 'zinc finger prote
998 input query terms found no hit:	['AF318333', 'AF318380', 'AF357221', 'AF400439', 'AF422192', 'AF426264', 'AF438406', 'AF452715', 'AF
1000 input query terms found no hit:	['ENST00000363937', 'ENST00000363944', 'ENST00000363947', 'ENST00000363948', 'ENST00000363957', 'ENS
1000 input query terms found no hit:	['ENST00000391269', 'ENST00000391270', 'ENST00000391274', 'ENST00000391295', 'ENST00000391297', 'ENS
1000 input query terms found no hit:	['ENST00000468013', 'ENST00000468131', 'ENST00000468140', 'ENST00000468169', 'ENST00000468197', 'ENS
997 input query terms found no hit:	['ENST00000522536', 'ENST00000522557', 'ENST00000522581', 'ENST00000522969', 'ENST00000522992', 'ENS
1000 input query terms found no hit:

In [46]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [47]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 249 samples.


In [48]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Craniosynostosis', the least common label is '0.0' with 50 occurrences. This represents 20.08% of the dataset.
The distribution of the feature 'Craniosynostosis' in this dataset is fine.

Quartiles for 'Age':
  25%: 5.23
  50% (Median): 8.77
  75%: 12.0
Min: 0.5
Max: 120.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 83 occurrences. This represents 33.33% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [49]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

In [50]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [14]:
# Check the information of usable cohorts
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH)
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE27976,True,True,False,True,True,249,


In [15]:
# If both age and gender have available cohorts, select 'age' as the condition.
condition = 'Age'
filter_column = 'has_' + condition.lower()

condition_best_cohort, condition_ranked_df = filter_and_rank_cohorts(JSON_PATH, filter_column)
condition_best_cohort

'GSE27976'

In [16]:
condition_ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,GSE27976,True,True,False,True,True,249,


In [17]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, condition_best_cohort + '.csv'))
merged_data.head()

,Craniosynostosis,Age,Gender,CA14,CD79A,EOS,FUSE,FAS,PARN,PTPRF,RCC1,TBP,TPR,U1,UCN,GLA,HASPIN,SP1,TESMIN
0,1.0,12.87,0.0,4.145010,8.201351,6.197000,9.854183,8.773215,4.893692,6.120276,7.702718,8.048879,8.591189,10.193450,11.66027,6.971736,6.117955,6.481451,5.188073
1,1.0,10.40,0.0,4.552640,8.226059,6.348450,10.059643,9.900642,5.082298,6.344258,7.599204,8.069599,8.504405,10.072650,11.35244,7.406352,6.598113,6.815678,5.241590
2,1.0,12.30,1.0,4.450482,8.327156,6.384476,9.926943,9.791214,4.701365,6.514661,8.280797,7.957904,8.378644,10.102500,11.70149,6.916776,5.854379,7.335926,5.095693
3,1.0,11.40,1.0,4.776999,7.865775,6.127192,9.815677,8.663057,4.891577,6.508160,7.443244,7.794022,8.456080,9.980424,11.88101,7.637656,6.259962,6.595198,5.481439
4,1.0,10.10,0.0,4.324631,8.490404,5.941068,9.929050,9.084288,4.666295,6.201364,7.960722,8.056880,8.520315,9.885635,11.05687,7.402481,6.050254,6.500354,5.156267


In [18]:
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [19]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [20]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [21]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 49.80% ± 3.56%


In [23]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

The cross-validation accuracy is 84.08% ± 2.71%


In [24]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


### 4. Discussion and report

In [25]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.1972
Found 14 genes with non-zero coefficients associated with the trait 'Craniosynostosis' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
Variable  Coefficient  Absolute Coefficient
    CA14     1.579909              1.579909
     UCN    -1.486378              1.486378
    FUSE    -1.438487              1.438487
     SP1    -1.015300              1.015300
   CD79A    -0.971751              0.971751
   PTPRF     0.896482              0.896482
     TPR     0.819390              0.819390
  HASPIN     0.815927              0.815927
     FAS     0.663122              0.663122
      U1    -0.530949              0.530949
    RCC1    -0.459531              0.459531
  TESMIN    -0.332096              0.332096
    PARN    -0.235098              0.235098
     EOS     0.036797              0.036797


/Users/ollie/Documents/JupyterProjects/AI4Science_gold_standard/stage1/Jinglin/utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [26]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.1972
Found 0 significant genes associated with the trait 'Craniosynostosis' conditional on the factor 'Age', with corrected p-value < 0.05:
Empty DataFrame
Columns: [Variable, Coefficient, p_value, corrected_p_value]
Index: []


/Users/ollie/Documents/JupyterProjects/AI4Science_gold_standard/stage1/Jinglin/utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
